In [2]:
## Helper functions

# Output
# this is needed to redirect the output to the screen. Else, it shows up in the log which is difficult to find
import os
import shutil


output = widgets.Output()
validation = False
uniquename = ""
local_ED2IN_file_path = ""
local_output_folder = ""
vars = {}

@output.capture()
def redirect_to_notebook(event):
    """    
    This function redirects the user to the next notebook, which displays the status of the job. It stores the hostname, username, and pkey values if provided, and opens the target notebook in a new window.
    """
    print("Going to next notebook")
    global hostname, username, job_id, pkey
    hostname = cluster_input.value
    username = username_input.value
    pkey = user_pkey.value
    if hostname:
        %store hostname
    if username:
        %store username
    if pkey:
        %store pkey
    target_url = "Show_job_status.ipynb"
    display(Javascript(f'window.open("{target_url}");'))

def auth_interactive_handler_callback(title, instructions, prompts):
    """
    Custom interactive handler callback for logging into HPC cluster for ED2_job_submission.
    """
    responses = []
    for prompt in prompts:
        prompt_text = prompt[0]
        echo = prompt[1]
        response = getpass.getpass(prompt_text) if echo else getpass.getpass('')
        responses.append(response)
    return responses

@output.capture()
def on_validate_button_click2(button):
    """
    Validates the input parameters for show_job_status.ipynb.
    """
    global validation2
    validation2 = False
    if username_input.value == "" or cluster_input.value == "" or job_id_input.value == "":
        print("Please enter a username, cluster, queue, job_id for the job you want to see the status")
        return
    
    validation2 = True
    print("Validation successful")

@output.capture()
def on_validate_button_click(button):
    """
    Validates the input parameters.
    """
    global validation
    validation = False
    if cluster_input.value == "localhost":
        validation = True
    elif username_input.value == "" or cluster_input.value == "" or queue_input.value == "" or job_name_input.value == "" or num_nodes_input.value == "" or runtime_input.value == "" or work_data_input.value == "":
        print("Please enter a username, cluster, queue, job name, number of nodes, runtime, and work folder")
        return
    else:
        validation = True
    print("Validation successful")

@output.capture()
def create_ED2IN_file(path_ED2IN, vars, output_folder):
    """
    Creates the ED2IN file with the specified variables.
    """
    
    local_file_path = output_folder + "/ED2IN" 

    # Read the existing ED2IN file
    with open(path_ED2IN, 'r') as f:
        lines = f.readlines()
    
    # Create a copy of the ED2IN file
    with open(local_file_path, 'w') as f:
        for line in lines:
            written = False
            for key, value in vars.items():
                if value != '' and key in line:
                    f.write(key + " = '" + value + "'\n")
                    written = True
                    break
            if not written:
                if "NL%FFILOUT" in line:
                    f.write("NL%FFILOUT = 'outputs/analysis'\n")
                elif "NL%SFILOUT" in line:
                    f.write("NL%SFILOUT = 'outputs/history'\n")
                else:
                    f.write(line)
    
    
    print("Created new ED2IN file at ", local_file_path)
    return local_file_path

@output.capture()
def createAndUpdateED2IN():
    if not validation:
        return
    path_ED2IN = ED2IN_path_input.value
    # vars to be replaced in ED2IN file
    global vars
    vars = {}
    for dropdown in var_dropdowns.children:
        var_option = dropdown.children[0].value
        var_value = dropdown.children[1].value
        vars[var_option] = var_value
    
    # local output folder
    global uniquename
    uniquename=generate_random_string()
    global local_output_folder
    local_output_folder=uniquename
    if not os.path.exists(local_output_folder):
        os.makedirs(local_output_folder)
        os.makedirs(local_output_folder + "/outputs")
    
    global local_ED2IN_file_path
    local_ED2IN_file_path = create_ED2IN_file(path_ED2IN, vars, local_output_folder)  
    
@output.capture()
def submitJob():
    """
    Submits a job for execution.

    This function submits a job for execution on a cluster or locally. It performs the following steps:
    1. Validates the input parameters.
    2. Executes the model locally if the hostname is 'localhost'.
    3. Sets up the necessary environment variables and commands for running the job on a cluster.
    4. Updates the ED2IN file with the specified variables.
    5. Creates a batch job script and transfers it to the cluster.
    6. Submits the batch job and extracts the job ID.
    7. Stores the job ID for future reference.

    Note: This function assumes that the necessary input parameters and variables have been set before calling it.

    Parameters:
    None

    Returns:
    None
    """
    if not validation:
        return
    print("Executing model")
    hostname = cluster_input.value


    if hostname == 'localhost':
        command = ['ed2', '-f', "ED2IN"]
        cwd = uniquename
        print(f"Starting to run model with {uniquename}/ED2IN")
        resource.setrlimit(resource.RLIMIT_STACK, (-1, -1))
        proc = subprocess.Popen(command, cwd=cwd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        for line in proc.stdout:
            print(line.decode(), end="")
        return

    username = username_input.value
    password = user_password.value
    pkey = user_pkey.value
    for cluster in clusters_data["clusters"]:
        if cluster["hostname"] == hostname:
            modules_to_load = cluster["modules_to_load"]
            pre_run_command = cluster["pre_run_command"]
            apptainer_binary_command = cluster["apptainer_binary_command"]
            post_run_command = cluster["post_run_command"]


    account = user_acc.value
    partition = queue_input.value
    job_name = job_name_input.value
    nodes = num_nodes_input.value
    time = runtime_input.value
    work_data = work_data_input.value
    path_singularity_image = ed_binary_singularity_input.children[1].value
    print("directory created:", uniquename)
    output_folder = uniquename
    job_name = "ED2IN-" + uniquename

    # Batch job details
    ntasks_per_node = 16                    # Number of task (cores/ppn) per node
    output = "openmp_" + job_name + ".o%j"  # Name of batch job output file
    error = "openmp_" + job_name + ".e%j"   # Name of batch job error file
    mail_user = username + "@illinois.edu"        # Send email notifications
    mail_type = "BEGIN,END"                 # Type of email notifications to send

    ssh_client = paramiko.SSHClient()
    ssh_client.load_system_host_keys()
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    connected = False
    
    print("Connecting to the cluster")
    if pkey != '':
        print("Using private key provided")
        key = paramiko.pkey.PKey.from_path(pkey)
        ssh_client.connect(hostname, username=username, pkey=key, allow_agent=True)
    else:
        print("Using password provided")
        try:
            ssh_client.connect(hostname, username=username, password=password, allow_agent=True)
            connected = True
            print("successfully connected")
        except:
            print("Failed to connect with the provided password. Trying interactive authentication.")
            pass
    transport = ssh_client.get_transport()
    if not connected:
        transport.auth_interactive(username=username, handler=auth_interactive_handler_callback)
    sftp_client = paramiko.SFTPClient.from_transport(transport)

    # Create the output folder and transfer the ED2IN file
    sftp_client.mkdir(output_folder)
    sftp_client.mkdir(output_folder + "/outputs")
    remote_ED2IN_path = f"{output_folder}/ED2IN"
    sftp_client.put(local_ED2IN_file_path, remote_ED2IN_path)
    #create the bat file
    with open(f"{local_output_folder}/{job_name}.sbatch", 'w') as f:
        f.writelines("#!/bin/bash\n")
        if account != '':
            f.writelines("#SBATCH --account=" + str(account) + "\n")
        f.writelines("#SBATCH --time=" + str(time) + "\n")
        #f.writelines("#SBATCH --nodes=" + str(nodes) + "\n")
        f.writelines("#SBATCH --ntasks-per-node=" + str(ntasks_per_node) + "\n")
        f.writelines("#SBATCH --job-name=" + job_name + "\n")
        f.writelines("#SBATCH --partition=" + partition + "\n")
        f.writelines("#SBATCH --output=" + output + "\n")
        f.writelines("#SBATCH --error=" + error + "\n")
        f.writelines("##SBATCH --mail-user=" + mail_user + "\n")
        f.writelines("##SBATCH --mail-type=" + mail_type + "\n")
        f.writelines("\n")
        f.writelines("\n")
        if modules_to_load != "":
            f.writelines("# load modules\n")
            f.writelines(f"module load {modules_to_load}" + "\n")
        if pre_run_command != "":
            f.writelines("# pre run command\n")
            f.writelines(f"{pre_run_command}" + "\n")
        bind_command = f"--bind $HOME/{output_folder}:/config --bind outputs:/data/outputs --bind {work_data}:/data{work_data}"
        for i, input_field in enumerate(additional_work_data_inputs):
            bind_command += f" --bind {input_field.value}:/data{input_field.value}"
        if apptainer_binary_command != "":
            f.writelines("# run apptainer\n")
            # Check if the singularity image exists
            f.writelines("echo 'Checking if Singularity image exists.'\n")
            f.writelines("if [ -f " + path_singularity_image + " ]; then\n")
            f.writelines("    echo 'Singularity image already exists.'\n")
            f.writelines(f"    {apptainer_binary_command} {bind_command}  --no-home --pwd /data {path_singularity_image} ed2 -f /config/ED2IN\n")
            f.writelines("else\n")
            f.writelines("    echo 'Singularity image does not exist. Pulling the image.'\n")
            f.writelines("    " + apptainer_binary_command.split()[0] + " pull ed2-gnu.sif docker://edmodel/ed2:gnu-PR-357\n")
            f.writelines(f"    {apptainer_binary_command} {bind_command}  --no-home --pwd /data ed2-gnu.sif ed2 -f /config/ED2IN")
        if post_run_command != "":
            f.writelines("# post run command\n")
            f.writelines(f"{post_run_command}" + "\n")
    f.close()

    #transfer .bat file to cluster and run it
    sftp_client.put(f"{local_output_folder}/{job_name}.sbatch", f"{output_folder}/{job_name}.sbatch")
    sftp_client.chmod(f"{output_folder}/{job_name}.sbatch", stat.S_IRWXU)
    _, stdo, stde = ssh_client.exec_command(f"cd {output_folder} && sbatch {job_name}.sbatch")
    print(stde.read().decode())

    # Extract the job ID from the sbatch output
    result = stdo.read().decode()
    print(result)
    submitted_job_id = result.split()[3]
    print(submitted_job_id)
    global job_id
    global remote_output_folder
    job_id=submitted_job_id
    remote_output_folder = output_folder
    if job_id:
        %store job_id
        %store remote_output_folder

    sftp_client.close()
    ssh_client.close()
    transport.close()

@output.capture()
def showJobStatus():
    """
    Connects to a remote server using SSH and retrieves the status of a job.

    This function connects to a remote server using SSH and retrieves the status of a job
    specified by the `job_id`. It requires the `hostname`, `username`, `password`, and `pkey`
    to establish the SSH connection. If a private key (`pkey`) is provided, it will be used
    for authentication; otherwise, the password will be used.

    Parameters:
    - None

    Returns:
    - None
    """
    hostname = cluster_input.value
    username = username_input.value
    password = user_password.value
    pkey = user_pkey.value
    output_folder = output_folder_input.value
    job_id = job_id_input.value
    ssh_client = paramiko.SSHClient()
    ssh_client.load_system_host_keys()
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    connected = False

    if pkey != '':
        print("Using private key provided")
        key = paramiko.pkey.PKey.from_path(pkey)
        ssh_client.connect(hostname, username=username, pkey=key, allow_agent=True)
    else:
        print("Using password provided")
        try:
            ssh_client.connect(hostname, username=username, password=password, allow_agent=True)
            connected = True
            print("successfully connected")
        except:
            print("Failed to connect with the provided password. Trying interactive authentication.")
            pass
    transport = ssh_client.get_transport()
    if not connected:
        transport.auth_interactive(username=username, handler=auth_interactive_handler_callback)
    sftp_client = paramiko.SFTPClient.from_transport(transport)

    # job status
    # Check the job status periodically
    print("Job status")
    while True:
        _, stdo, stde = ssh_client.exec_command(f"squeue -u {username} -j {job_id}")
        job_status = stdo.read().decode()
        print(job_status)

        # Break the loop if the job is completed or failed
        if job_id not in job_status:
            break

        # Wait for a few seconds before checking again
        timer.sleep(30)

    print("Output")
    # View output
    try:
        _, stdo, stde = ssh_client.exec_command(f"cat {output_folder}/*.o{job_id}")
        print(stdo.read().decode())
        print(stde.read().decode())
    except:
        print("No output file found")
    
    print("Error")
    # View error
    try:
        _, stdo, stde = ssh_client.exec_command(f"cat {output_folder}/*.e{job_id}")
        print(stdo.read().decode())
        print(stde.read().decode())
    except:
        print("No error file found")
    
    print("Copying output files here")
    files = sftp_client.listdir(output_folder + "/outputs")
    
    # Ensure local directory exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Download each file
    for file in files:
        remote_filepath = os.path.join(output_folder + "/outputs", file)
        local_filepath = os.path.join(output_folder + "/outputs", file)
        try:
            sftp_client.get(remote_filepath, local_filepath)
            print(f"Downloaded {file} to {local_filepath}")
        except Exception as e:
            print(f"Failed to download {file}: {e}")

    sftp_client.close()
    ssh_client.close()
    transport.close()

def handle_cluster_change(change):
    """
    Handles changes in the cluster dropdown.
    """
    selected_cluster = change.new

    # Disable or enable widgets based on the selected cluster
    if selected_cluster == "localhost":
        username_input.disabled = True
        batch_job_input.disabled = True
        user_password.disabled = True
        user_acc.disabled = True
        queue_input.disabled = True
        ed_binary_singularity_input.children[1].disabled = True
        job_name_input.disabled = True
        num_nodes_input.disabled = True
        runtime_input.disabled = True
        work_data_input.disabled = True
        additional_work_data_inputs = []
        add_work_data_button.disabled = True
        remove_work_data_button.disabled = True
        user_pkey.disabled = True
    else:
        username_input.disabled = False
        username_input.value = ""
        user_password.disabled = False
        user_password.value = ""
        batch_job_input.options = batch_jobs_dict.get(selected_cluster, [])
        queue_input.options = queues_dict.get(selected_cluster, [])
        batch_job_input.disabled = False
        user_acc.disabled = False
        user_acc.value = ""
        queue_input.disabled = False
        ed_binary_singularity_input.children[1].disabled = False
        job_name_input.disabled = False
        num_nodes_input.disabled = False
        num_nodes_input.value = 1
        runtime_input.disabled = False
        runtime_input.value = "00:15:00"
        work_data_input.disabled = False
        additional_work_data_inputs = []
        add_work_data_button.disabled = False
        remove_work_data_button.disabled = False
        user_pkey.disabled = False


#<-------------------------UI related helper functions--------------------------------------------------->
def generate_random_string(length=4, chars="abcdefghijklmnopqrstuvwxyz0123456789"):
    """
    Generates a short random name.

    Args:
        length (int): The length of the random name. Default is 4.
        chars (str): The characters to choose from for generating the random name. Default is lowercase letters and digits.

    Returns:
        str: The generated random name.
    """
    return "".join(random.choice(chars) for _ in range(length))

def add_work_data_input(b):
    """
    Adds a new work data input widget to the existing list of additional work data inputs.

    Parameters:
    - b: The button object that triggers the addition of a new work data input.

    Returns:
    None
    """
    new_input = widgets.Text(placeholder="Work data input", description=f"Work data {len(additional_work_data_inputs) + 2}:")
    additional_work_data_inputs.append(new_input)
    work_data_box.children = tuple(additional_work_data_inputs)

def remove_work_data_input(b):
    """
    Removes the last additional work data input from the list and updates the work_data_box.
    
    Args:
        b: The button object that triggered the function.
    """
    if additional_work_data_inputs:
        additional_work_data_inputs.pop()
        work_data_box.children = tuple(additional_work_data_inputs)


def add_dropdown(button):
    """
    Handles the addition of a new dropdown widget.
    """
    # Remove selected options from var_options
    for dropdown in var_dropdowns.children:
        selected_option = dropdown.children[0].value
        if selected_option in var_options:
            var_options.remove(selected_option)

    # Add the new dropdown widget
    var_dropdowns.children += (create_dropdown(),)
    if len(var_options) == 1:
        add_button.disabled = True

def create_dropdown():
    """
    Creates a new dropdown widget.

    Returns:
        ipywidgets.Dropdown: The created dropdown widget.
    """
    # Code for creating the dropdown widget
    return widgets.HBox([widgets.Dropdown(options=var_options, description='Replace:'), widgets.Text(placeholder="Enter the path")])

def remove_dropdown(button):
    ''' Function to remove a dropdown widget'''
    if len(var_dropdowns.children) >= 1:
        var_dropdowns.children = var_dropdowns.children[:-1]
        var_option = var_dropdowns.children[-1].children[0].value
        if var_option not in var_options:
            var_options.append(var_option)
        add_button.disabled = False


NameError: name 'widgets' is not defined